# Проект для «Викишоп»

Интернет-магазин **«Викишоп»** запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.
Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

**План работы:**
- Загрузить и подготовить данные.
- Обучить разные модели.
- Общие выводы.

##Загрузка данных

In [ ]:

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import nltk
from nltk.corpus import stopwords as nltk_stopwords
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english')) ###
nltk.download('punkt_tab')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger_eng')
from sklearn.linear_model import LogisticRegression
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV
import spacy
from tqdm.notebook import tqdm
from sklearn.linear_model import PassiveAggressiveClassifier

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


In [ ]:
#const
RANDOM_STATE = 42
pd.set_option('display.max_columns', None)
tqdm.pandas()

In [ ]:
df = pd.read_csv(r'/datasets/toxic_comments.csv', index_col = 0)

In [ ]:
df.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


проверим на пропуски (на всякий случай)

In [ ]:
df.isna().sum()

,0
text,0
toxic,0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 159292 entries, 0 to 159450
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.6+ MB


In [ ]:
df['toxic'].value_counts()

,count
toxic,
0,143106
1,16186


Виден явный дизбаланс классов, для начала обучу простую log reg и оценим ее f1 меру. Дальше буду отталкиваться от этого значения (можно написать пайплайн, где буду просто перебирать с помощью cv все модели и гиперпараметры в надежде получить метрику > 0.75 или дполонительно переберу разные методы борьбы с дизбалансом)

для начала лемматизируем комм-рии

In [ ]:

lemmatizer = WordNetLemmatizer()
def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

def lemmatize_text(text):
    words = nltk.word_tokenize(text)
    pos_tags = nltk.pos_tag(words)
    lemmatized_words = []
    for word, pos_tag in pos_tags:
        wordnet_pos = get_wordnet_pos(pos_tag)
        lemma = lemmatizer.lemmatize(word, pos=wordnet_pos)
        lemmatized_words.append(lemma)
    return ' '.join(lemmatized_words)
sentence1 = "The striped bats are hanging on their feet for best" # проверим на всякий
sentence2 = "you should be ashamed of yourself went worked"
df_my = pd.DataFrame([sentence1, sentence2], columns = ['text'])
print(df_my['text'].apply(lemmatize_text))


0    The striped bat be hang on their foot for best
1         you should be ashamed of yourself go work
Name: text, dtype: object


In [ ]:
df['lemmatized_text'] = df['text'].progress_apply(lemmatize_text) #леммаатизируем весь дата сет
df.head()

  0%|          | 0/159292 [00:00<?, ?it/s]

,text,toxic,lemmatized_text
0,Explanation\nWhy the edits made under my usern...,0,Explanation Why the edits make under my userna...
1,D'aww! He matches this background colour I'm s...,0,D'aww ! He match this background colour I 'm s...
2,"Hey man, I'm really not trying to edit war. It...",0,"Hey man , I 'm really not try to edit war . It..."
3,"""\nMore\nI can't make any real suggestions on ...",0,`` More I ca n't make any real suggestion on i...
4,"You, sir, are my hero. Any chance you remember...",0,"You , sir , be my hero . Any chance you rememb..."


In [ ]:
df.drop('text', axis = 1, inplace = True)

In [ ]:
X_train,X_test,target_train,target_test = train_test_split(df.drop('toxic', axis = 1), df['toxic'],
                                                           stratify = df['toxic'], random_state = RANDOM_STATE) #stratify - сохраним соотношения классов

In [ ]:
# считаем величину TF-IDF для обучающей выборки
count_tf_idf_train = TfidfVectorizer(stop_words = list(stopwords))
tf_idf_train = count_tf_idf_train.fit_transform(X_train.lemmatized_text)
display(tf_idf_train.shape)

(119469, 152080)

In [ ]:
# считаем величину TF-IDF для тестовой выборки
count_tf_idf_test = TfidfVectorizer(stop_words = list(stopwords))
tf_idf_testing = count_tf_idf_train.transform(X_test.lemmatized_text)
tf_idf_test, tf_idf_valid = train_test_split(tf_idf_testing, test_size = 0.5, random_state = RANDOM_STATE)
y_test, y_valid = train_test_split(target_test , test_size = 0.5, random_state = RANDOM_STATE)
display(tf_idf_test.shape)

(19911, 152080)

##Обучение моделей

In [ ]:
# задаем алгоритм для модели
model = LogisticRegression(random_state= RANDOM_STATE)

In [ ]:
# обучаем модель
model.fit(tf_idf_train, target_train)

LogisticRegression(random_state=42)

In [ ]:
# pred
pred_test = model.predict(tf_idf_valid)
print('f1: {:.2f}'.format(f1_score(y_valid, pred_test)))

f1: 0.72


Стоковая модель дала метрику f1 со значением 0.72

Попробуем улучшить это великолепное значение путем решения проблемы дизбаланса классов.

Начнем с увеличения весов toxic (хотя при таком сильном неравенстве классов это врядли поможет)

In [ ]:
model_2 = LogisticRegression(class_weight='balanced', solver='lbfgs', random_state = RANDOM_STATE)
model_2.fit(tf_idf_train, target_train)
pred_test = model_2.predict(tf_idf_valid)
print('f1: {:.2f}'.format(f1_score(y_valid, pred_test)))

f1: 0.74


метрика увеличилась на ДВЕ сотых - 0.74

попробуем добавить регуляризацию

In [ ]:
par_grid = {'penalty':['l1','l2'], 'C':[0.01,0.1,1,5,6,7,8,9,10,100]}

In [ ]:
model_logres_cv = LogisticRegression(random_state = RANDOM_STATE,
                                     class_weight='balanced', solver='liblinear')
                                     # можно добавить max_iter чтобы сходилось, но время перебора в разы возрастет
cv = GridSearchCV(model_logres_cv, param_grid = par_grid, scoring = 'f1')
cv.fit(tf_idf_train, target_train)

GridSearchCV(estimator=LogisticRegression(class_weight='balanced',
                                          random_state=42, solver='liblinear'),
             param_grid={'C': [0.01, 0.1, 1, 5, 6, 7, 8, 9, 10, 100],
                         'penalty': ['l1', 'l2']},
             scoring='f1')

In [ ]:
print(f'Наилучшее значение f1 меры после cv {cv.best_score_}')

Наилучшее значение f1 меры после cv 0.7613088560289507


In [ ]:
print(f'Гиперпараметры полученной модели {cv.best_params_}')

Гиперпараметры полученной модели {'C': 8, 'penalty': 'l2'}


Пробуем PassiveAggressiveClassifier

In [ ]:
pac = PassiveAggressiveClassifier(random_state=RANDOM_STATE, max_iter=100)
pac.fit(tf_idf_train, target_train)
ypred = pac.predict(tf_idf_valid)
f1 = f1_score(y_valid, ypred)
print(f'f1 мера - {round(f1,2)}')

f1 мера - 0.73


к сожалению пассивно-агрессивный классификатор дал метрику хуже(однако спасибо я почитаю про нее в дальнейшем)

получаем f1 > 0.75, время проверить на тестовой

In [ ]:
final_model = cv.best_estimator_

In [ ]:
f1_score(y_test,final_model.predict(tf_idf_test))

0.7629310344827587

##Общий вывод
- Загрузили данные (160.000 +- комментариев)
- пропусков нет, провели лемматизацию
- стоковая модель LogisticREgression дала значение метрики f1 0.72+-0.01
- стоковая модель PassiveAggressiveClassifier - f1 - 0.73
- с помощью cv перебрали losso и ridge, в том числе и параметр C ->{'C': 7, 'penalty': 'l2' и получили необходимую метрику

**Задача выполнена значение f1 меры > 0.75**